In [ ]:
import datetime
import pytz


%run ./common.ipynb

In [ ]:
# searchQry='?jql=key%20in%20(%27CSP-151%27,%27CSP-963%27)&expand=changelog'
# searchQry='?jql=sprint%20=%207431%20and%20type%20=%20Story&expand=changelog'
searchQry='?jql= project in ("CSP") and type  in (Story) and updated <= startofday(-90d)&fields=key,status&expand=changelog&maxResults=5000'
cycleTimeDf = pd.DataFrame(columns=['key','leadTime','development','QA','deployment'])
searchQry_DS = getDataSet(searchQry)

In [ ]:
changelog_Rec =[ ]
for searchResult in searchQry_DS:
    changelog = searchResult['changelog']
    for history in changelog['histories']:
        history_id = history['id']
        created_at = history.get('created',None)
        for item in history['items']:
            # print(item_list['field'])
            changelog_Rec.append({
                'key': searchResult['key'],
                'created_at': created_at,
                'field': item['field'],
                'from': item['fromString'],
                'to': item['toString'],
            })

changelog_Rec_df = pd.DataFrame(changelog_Rec)

status_change = changelog_Rec_df[changelog_Rec_df['field']=='status']

status_change = status_change.reset_index(drop=True)


In [ ]:
def calculateCycleTime(_status_change,current_status,key):
    cycleTime = {}
    if (current_status != 'Done'):
        # Get the current time in UTC
        now_utc = datetime.datetime.now(pytz.utc)
        # Convert to Eastern Time
        eastern = pytz.timezone('US/Eastern')
        now_est = now_utc.astimezone(eastern)
        new_node = {
                    'key':key,
                    'created_at': now_est.strftime('%Y-%m-%dT%H:%M:%S.%f%z'),
                    'field': 'status',
                    'from': current_status,
                    'to' : 'Active'
                    }
        _status_change.loc[len(_status_change)] = new_node     

    _status_change = _status_change.reset_index(drop=True)
    _status_change['created_at'] = pd.to_datetime(_status_change['created_at'], format='%Y-%m-%dT%H:%M:%S.%f%z')
    _status_change['time_diff'] = _status_change['created_at'].diff()
    _status_change = _status_change.reset_index(drop=True)
    cycleTime['key'] = _status_change['key'][0]
    cycleTime['leadTime'] = _status_change['time_diff'].sum()
    cycleTime['development'] = _status_change[_status_change['to'].isin( ['In Progress','Code Review','Testing','Active'])]['time_diff'].sum()
    cycleTime['QA'] = _status_change[_status_change['from'].isin( ['Testing']) ] [_status_change['to'].isin( ['Deploy', 'Active'])]['time_diff'].sum()
    cycleTime['deployment'] = _status_change[_status_change['from'].isin( ['Deploy']) ]['time_diff'].sum()

    display (_status_change)
    display(cycleTime)
    return cycleTime


In [ ]:
for searchResult in searchQry_DS:
    result = status_change[status_change['key'] == searchResult['key']]
    cycleTimeDf = cycleTimeDf.append(calculateCycleTime(result,searchResult['fields']['status']['name'],searchResult['key']), ignore_index=True)

display(cycleTimeDf)
